In [1]:
import pandas as pd, requests, pymongo, psycopg2, numpy as np
import pandas.io.sql as sqlio

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)



# Crime Data

In [8]:
response = requests.get("https://data.lacity.org/api/views/63jg-8b9z/rows.json?accessType=DOWNLOAD")


In [ ]:
# myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# mydb = myclient["DAP_souvik"]
# mycol = mydb["crime"]

In [9]:
all_data =response.json()
cols = [col_name["name"] for col_name in all_data["meta"]["view"]["columns"]]

In [10]:
# all_data =response.json()
# making key-value pairs
values=list(map(list, zip(*[data for data in all_data["data"]])))


In [11]:
keyvalues =[dict(zip(cols, item)) for item in zip(*values)]

In [12]:
df = pd.DataFrame(keyvalues)

In [37]:
df.head()

,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,investigation,adult_other,adult_arrest,juvenile_arrest,VD_B,VD_W,VD_A,VD_H,VD_O
0,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,None,AA,Adult Arrest,0,0,1,0,0,0,0,1,0
1,2010-09-12,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,101.0,STREET,NaN,None,IC,Invest Cont,1,0,0,0,0,1,0,0,0
2,2010-08-09,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0,M,H,103.0,ALLEY,NaN,None,IC,Invest Cont,1,0,0,0,0,0,0,1,0
3,2010-01-05,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,1,0,0,0,0,1,0,0,0
4,2010-01-02,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,1,0,0,0,0,0,0,1,0


In [14]:
df.columns

Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC',
       'AREA ', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd',
       'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent',
       'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status',
       'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
       'LOCATION', 'Cross Street', 'LAT', 'LON'],
      dtype='object')

In [15]:
df.drop(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'DR_NO', 'Date Rptd', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
       'LOCATION', 'Cross Street', 'LAT', 'LON'],axis=1,inplace=True)

In [16]:
df.drop(['Mocodes'],axis=1,inplace=True)

In [173]:
df['Vict Age'].isna().sum()
df['Vict Age'].fillna(0,inplace=True)

In [174]:
df['Vict Age']=pd.to_numeric(df['Vict Age'])
n = int(sum(df['Vict Age'])/len(df['Vict Age']))
df['Vict Age'].replace(0,35,inplace=True)

In [175]:
limit=[]
for i in df['Vict Age']:
    if i<19:
        limit.append('0-18')
    elif i<31:
        limit.append('19-30')
    elif i<46:
        limit.append('31-45')
    elif i<61:
        limit.append('46-60')
    else:
        limit.append('60+')
df['Age group']=limit

In [176]:
df['Age group'].value_counts()

31-45    907229
19-30    494490
46-60    369548
60+      169944
0-18     114853
Name: Age group, dtype: int64

In [178]:
df1=pd.get_dummies(df['Age group'])

In [179]:
df1.head()

,0-18,19-30,31-45,46-60,60+
0,0,0,0,1,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0


In [180]:
df['0-18']=df1['0-18']
df['19-30']=df1['19-30']
df['31-45']=df1['31-45']
df['46=60']=df1['46-60']
df['60+']=df1['60+']

In [181]:
df.head()

,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,investigation,adult_other,adult_arrest,juvenile_arrest,VD_B,VD_W,VD_A,VD_H,VD_O,Male,Female,weapon used,hr,min,Age group,0-18,19-30,31-45,46=60,60+
0,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,None,AA,Adult Arrest,0,0,1,0,0,0,0,1,0,1,0,1,13.50,50,46-60,0,0,0,1,0
1,2010-09-12,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",35,M,W,101.0,STREET,NaN,None,IC,Invest Cont,1,0,0,0,0,1,0,0,0,1,0,1,0.45,45,31-45,0,0,1,0,0
2,2010-08-09,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,35,M,H,103.0,ALLEY,NaN,None,IC,Invest Cont,1,0,0,0,0,0,0,1,0,1,0,1,15.15,15,31-45,0,0,1,0,0
3,2010-01-05,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,1,0,0,0,0,1,0,0,0,0,1,1,1.50,50,46-60,0,0,0,1,0
4,2010-01-02,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,1,0,0,0,0,0,0,1,0,0,1,1,21.00,0,46-60,0,0,0,1,0


In [182]:
# converting required columns to numeric type
for i in  df.columns:
    try:
        df[i]=pd.to_numeric(df[i])
    except:
        pass

In [233]:
df['DATE OCC']=pd.to_datetime(df['DATE OCC'])

In [234]:
df.dtypes

DATE OCC           datetime64[ns]
TIME OCC                    int64
AREA                        int64
AREA NAME                  object
Rpt Dist No                 int64
Part 1-2                    int64
Crm Cd                      int64
Crm Cd Desc                object
Vict Age                    int64
Vict Sex                   object
Vict Descent               object
Premis Cd                 float64
Premis Desc                object
Weapon Used Cd            float64
Weapon Desc                object
Status                     object
Status Desc                object
investigation               uint8
adult_other                 uint8
adult_arrest                uint8
juvenile_arrest             uint8
VD_B                        uint8
VD_W                        uint8
VD_A                        uint8
VD_H                        uint8
VD_O                        uint8
Male                        uint8
Female                      uint8
weapon used                 int64
hr            

In [41]:
df=df[df['Status'].isin(['IC','AO','AA','JA'])]

In [24]:
df['Status Desc'].value_counts()

Invest Cont     1587396
Adult Other      244103
Adult Arrest     210365
Juv Arrest        14200
Name: Status Desc, dtype: int64

In [25]:
df['Weapon Used Cd'].value_counts()

400.0    417620
500.0     58541
511.0     56941
102.0     34289
109.0     13387
200.0     13076
207.0      9733
106.0      8838
307.0      7159
101.0      5182
212.0      5150
308.0      4625
306.0      4605
512.0      4346
304.0      4325
204.0      4313
302.0      4016
205.0      3799
201.0      3493
113.0      3416
114.0      2882
312.0      2713
301.0      2409
218.0      2228
515.0      1809
311.0      1666
219.0      1606
223.0      1429
215.0      1325
216.0      1143
104.0      1109
107.0      1005
310.0       973
103.0       859
305.0       851
513.0       676
506.0       627
309.0       596
206.0       552
501.0       515
221.0       479
503.0       473
303.0       423
514.0       360
211.0       343
112.0       311
510.0       300
504.0       296
210.0       245
217.0       204
208.0       188
108.0       175
115.0       165
505.0       161
214.0       161
122.0       160
203.0       136
105.0       132
507.0       129
111.0        98
509.0        91
110.0        90
213.0   

In [27]:
df4= pd.get_dummies(df['Status'])
df4.head()

,AA,AO,IC,JA
0,1,0,0,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


In [ ]:
df['investigation']=df4['IC']
df['adult_other']=df4['AO']
df['adult_arrest']=df4['AA']
df['juvenile_arrest']=df4['JA']

In [170]:
df1= pd.get_dummies(df['Vict Sex'])

In [171]:
df1.head()

,-,F,H,M,N,X
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0


In [30]:
df1= pd.get_dummies(df['Vict Descent'])

In [31]:
df1.head()

,-,A,B,C,D,F,G,H,I,J,K,L,O,P,S,U,V,W,X,Z
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
df['VD_B']=df1['B']
df['VD_W']=df1['W']
df['VD_A']=df1['A']
df['VD_H']=df1['H']
df['VD_O']=df1['O']


In [43]:
df1= pd.get_dummies(df['Vict Sex'])
df['Male']=df1['M']
df['Female']=df1['F']

In [44]:
# weapon used or not
df['weapon used'] = list(map(lambda x: 0 if x==0 else 1, df['Weapon Desc']))

In [45]:
df['weapon used'].value_counts()

1    2056064
Name: weapon used, dtype: int64

In [235]:
df['hr']=df['TIME OCC']/100
df['hr'].astype(int)

0          13
1           0
2          15
3           1
4          21
           ..
2060943     4
2060944    18
2060945    21
2060946    18
2060947    16
Name: hr, Length: 2056064, dtype: int32

In [236]:
df['min']=df['TIME OCC']%100
df['min'].astype(int)

0          50
1          45
2          15
3          50
4           0
           ..
2060943     0
2060944    10
2060945     0
2060946     0
2060947    15
Name: min, Length: 2056064, dtype: int32

In [237]:
df.head()

,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,investigation,adult_other,adult_arrest,juvenile_arrest,VD_B,VD_W,VD_A,VD_H,VD_O,Male,Female,weapon used,hr,min,Age group,0-18,19-30,31-45,46=60,60+
0,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,48,M,H,501.0,SINGLE FAMILY DWELLING,0.0,0,AA,Adult Arrest,0,0,1,0,0,0,0,1,0,1,0,1,13.50,50,46-60,0,0,0,1,0
1,2010-09-12,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",35,M,W,101.0,STREET,0.0,0,IC,Invest Cont,1,0,0,0,0,1,0,0,0,1,0,1,0.45,45,31-45,0,0,1,0,0
2,2010-08-09,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,35,M,H,103.0,ALLEY,0.0,0,IC,Invest Cont,1,0,0,0,0,0,0,1,0,1,0,1,15.15,15,31-45,0,0,1,0,0
3,2010-01-05,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,1,0,0,0,0,1,0,0,0,0,1,1,1.50,50,46-60,0,0,0,1,0
4,2010-01-02,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,1,0,0,0,0,0,0,1,0,0,1,1,21.00,0,46-60,0,0,0,1,0


In [238]:
ddf=df.groupby('DATE OCC', as_index=False).agg({'Status Desc':'count','hr': np.mean,'AREA NAME':pd.Series.mode,'AREA ':pd.Series.mode,
                                              'Crm Cd':pd.Series.mode, 'Crm Cd Desc':pd.Series.mode,'Vict Age':np.median, 
                                                'Premis Desc':pd.Series.mode, 'Premis Cd':pd.Series.mode,
                                               'weapon used':sum, 'Male':sum,'Female':sum,'VD_B':sum,'VD_W':sum,'VD_A':sum,'VD_H':sum,'VD_O':sum,'weapon used':sum,'investigation':sum,'adult_other':sum,'adult_arrest':sum, 'juvenile_arrest':sum,'0-18':sum,'19-30':sum,'31-45':sum,'46=60':sum, '60+':sum,})

In [162]:
# df['Weapon Used Cd'].value_counts()

In [240]:
len(ddf.columns)
ddf.columns

Index(['DATE OCC', 'Status Desc', 'hr', 'AREA NAME', 'AREA ', 'Crm Cd',
       'Crm Cd Desc', 'Vict Age', 'Premis Desc', 'Premis Cd', 'weapon used',
       'Male', 'Female', 'VD_B', 'VD_W', 'VD_A', 'VD_H', 'VD_O',
       'investigation', 'adult_other', 'adult_arrest', 'juvenile_arrest',
       '0-18', '19-30', '31-45', '46=60', '60+'],
      dtype='object')

In [241]:
df.fillna(0,inplace=True)

In [242]:
ddf.shape

(4353, 27)

In [243]:
ddf['hr']=ddf['hr'].astype(int)
ddf['weapon used']=ddf['weapon used']/ddf['Status Desc'] 
ddf['Male']=ddf['Male']/ddf['Status Desc']
ddf['Female']=ddf['Female']/ddf['Status Desc']
ddf['VD_B']=ddf['VD_B']/ddf['Status Desc']
ddf['VD_W']=ddf['VD_W']/ddf['Status Desc']
ddf['VD_A']=ddf['VD_A']/ddf['Status Desc']
ddf['VD_H']=ddf['VD_H']/ddf['Status Desc']
ddf['VD_O']=ddf['VD_O']/ddf['Status Desc']
ddf['investigation']=ddf['investigation']/ddf['Status Desc']
ddf['adult_other']=ddf['adult_other']/ddf['Status Desc']
ddf['adult_arrest']=ddf['adult_arrest']/ddf['Status Desc']
ddf['juvenile_arrest']=ddf['juvenile_arrest']/ddf['Status Desc']

ddf.head(10)

,DATE OCC,Status Desc,hr,AREA NAME,AREA,Crm Cd,Crm Cd Desc,Vict Age,Premis Desc,Premis Cd,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,Mission,19,354,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,501.0,1.0,0.418403,0.546441,0.156684,0.183160,0.018663,0.501736,0.093316,0.831163,0.107205,0.057292,0.004340,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,Mission,19,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,1.0,0.476636,0.414953,0.155140,0.244860,0.020561,0.357009,0.091589,0.824299,0.084112,0.085981,0.005607,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,77th Street,12,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,1.0,0.459108,0.429368,0.156134,0.263941,0.011152,0.373606,0.078067,0.832714,0.065056,0.096654,0.005576,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,77th Street,12,"[330, 624]","[BATTERY - SIMPLE ASSAULT, BURGLARY FROM VEHICLE]",35.0,STREET,101.0,1.0,0.439856,0.448833,0.156194,0.247756,0.023339,0.351885,0.087971,0.856373,0.070018,0.070018,0.003591,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,Southwest,3,310,BURGLARY,35.0,SINGLE FAMILY DWELLING,501.0,1.0,0.440585,0.446069,0.168190,0.288848,0.023766,0.308958,0.084095,0.840951,0.085923,0.069470,0.003656,35.0,127.0,239.0,106.0,40.0
5,2010-01-06,532,14,"[Southwest, West Valley]","[3, 10]","[310, 330]","[BURGLARY, BURGLARY FROM VEHICLE]",35.0,"[SINGLE FAMILY DWELLING, STREET]","[101.0, 501.0]",1.0,0.507519,0.404135,0.159774,0.293233,0.009398,0.321429,0.092105,0.808271,0.095865,0.093985,0.001880,28.0,126.0,246.0,91.0,41.0
6,2010-01-07,550,13,Southwest,3,310,BURGLARY,35.0,SINGLE FAMILY DWELLING,501.0,1.0,0.460000,0.441818,0.161818,0.247273,0.014545,0.372727,0.089091,0.841818,0.072727,0.080000,0.005455,37.0,139.0,249.0,85.0,40.0
7,2010-01-08,605,14,Southwest,3,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,1.0,0.502479,0.386777,0.188430,0.242975,0.018182,0.347107,0.080992,0.829752,0.082645,0.082645,0.004959,39.0,133.0,278.0,105.0,50.0
8,2010-01-09,631,13,Southwest,3,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,1.0,0.483360,0.419968,0.129952,0.267829,0.030111,0.358162,0.095087,0.833597,0.068146,0.093502,0.004754,46.0,192.0,262.0,97.0,34.0
9,2010-01-10,617,12,77th Street,12,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,1.0,0.489465,0.414911,0.152350,0.262561,0.019449,0.367909,0.094003,0.847650,0.082658,0.068071,0.001621,54.0,180.0,245.0,104.0,34.0


In [244]:
ddf.rename(columns = {'Status Desc':'Crime_count'}, inplace = True)

In [246]:
ddf.drop(['AREA NAME','Crm Cd','Premis Cd'],axis=1,inplace=True)

In [247]:
ddf.head()

,DATE OCC,Crime_count,hr,AREA,Crm Cd Desc,Vict Age,Premis Desc,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,19,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,1.0,0.418403,0.546441,0.156684,0.183160,0.018663,0.501736,0.093316,0.831163,0.107205,0.057292,0.004340,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,19,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.476636,0.414953,0.155140,0.244860,0.020561,0.357009,0.091589,0.824299,0.084112,0.085981,0.005607,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,12,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.459108,0.429368,0.156134,0.263941,0.011152,0.373606,0.078067,0.832714,0.065056,0.096654,0.005576,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,12,"[BATTERY - SIMPLE ASSAULT, BURGLARY FROM VEHICLE]",35.0,STREET,1.0,0.439856,0.448833,0.156194,0.247756,0.023339,0.351885,0.087971,0.856373,0.070018,0.070018,0.003591,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,3,BURGLARY,35.0,SINGLE FAMILY DWELLING,1.0,0.440585,0.446069,0.168190,0.288848,0.023766,0.308958,0.084095,0.840951,0.085923,0.069470,0.003656,35.0,127.0,239.0,106.0,40.0


In [248]:
ddf.to_csv("final_crimeData.csv",index=False)

In [249]:
ddf = pd.read_csv("final_crimeData.csv")

In [250]:
ddf.head()

,DATE OCC,Crime_count,hr,AREA,Crm Cd Desc,Vict Age,Premis Desc,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,19,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,1.0,0.418403,0.546441,0.156684,0.183160,0.018663,0.501736,0.093316,0.831163,0.107205,0.057292,0.004340,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,19,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.476636,0.414953,0.155140,0.244860,0.020561,0.357009,0.091589,0.824299,0.084112,0.085981,0.005607,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,12,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.459108,0.429368,0.156134,0.263941,0.011152,0.373606,0.078067,0.832714,0.065056,0.096654,0.005576,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,12,['BATTERY - SIMPLE ASSAULT' 'BURGLARY FROM VEH...,35.0,STREET,1.0,0.439856,0.448833,0.156194,0.247756,0.023339,0.351885,0.087971,0.856373,0.070018,0.070018,0.003591,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,3,BURGLARY,35.0,SINGLE FAMILY DWELLING,1.0,0.440585,0.446069,0.168190,0.288848,0.023766,0.308958,0.084095,0.840951,0.085923,0.069470,0.003656,35.0,127.0,239.0,106.0,40.0


In [113]:
# all_data =response.json()
# cols = [col_name["name"] for col_name in all_data["meta"]["view"]["columns"]]

In [5]:
# key-values pair
# all_data =response.json()
# values=list(map(list, zip(*[data for data in all_data["data"]])))



In [6]:
# keyvalues =[dict(zip(cols, item)) for item in zip(*values)]

In [7]:
# inserted data 
# many_ids = mycol.insert_many(keyvalues).inserted_ids
# df = pd.DataFrame(keyvalues)

In [9]:
# fetching data from mongodb
# dff=[]
# for i in mycol.find():
#     dff.append(i)
    
# df = pd.DataFrame(dff)

In [114]:
# df

In [ ]:
# df.drop(["_id", "sid", "id", "position", "created_at", "created_meta", "created_at", "updated_meta", "updated_at", "meta"], axis=1, inplace=True)

In [19]:
import psycopg2
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "postgres")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE DAP;')
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL step 1", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()


In [172]:
# ddf.columns

In [251]:
ddf=pd.DataFrame(ddf)
ddf.head()

,DATE OCC,Crime_count,hr,AREA,Crm Cd Desc,Vict Age,Premis Desc,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,19,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,1.0,0.418403,0.546441,0.156684,0.183160,0.018663,0.501736,0.093316,0.831163,0.107205,0.057292,0.004340,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,19,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.476636,0.414953,0.155140,0.244860,0.020561,0.357009,0.091589,0.824299,0.084112,0.085981,0.005607,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,12,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.459108,0.429368,0.156134,0.263941,0.011152,0.373606,0.078067,0.832714,0.065056,0.096654,0.005576,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,12,['BATTERY - SIMPLE ASSAULT' 'BURGLARY FROM VEH...,35.0,STREET,1.0,0.439856,0.448833,0.156194,0.247756,0.023339,0.351885,0.087971,0.856373,0.070018,0.070018,0.003591,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,3,BURGLARY,35.0,SINGLE FAMILY DWELLING,1.0,0.440585,0.446069,0.168190,0.288848,0.023766,0.308958,0.084095,0.840951,0.085923,0.069470,0.003656,35.0,127.0,239.0,106.0,40.0


In [252]:
ddf.columns


Index(['DATE OCC', 'Crime_count', 'hr', 'AREA ', 'Crm Cd Desc', 'Vict Age',
       'Premis Desc', 'weapon used', 'Male', 'Female', 'VD_B', 'VD_W', 'VD_A',
       'VD_H', 'VD_O', 'investigation', 'adult_other', 'adult_arrest',
       'juvenile_arrest', '0-18', '19-30', '31-45', '46=60', '60+'],
      dtype='object')

In [253]:
ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("\'","")
ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("[","")
ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("]","")

ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("\'","")
ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("[","")
ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("]","")


ddf['AREA ']=ddf['Crm Cd Desc'].str.replace("\'","")
ddf['AREA ']=ddf['Crm Cd Desc'].str.replace("[","")
ddf['AREA ']=ddf['Crm Cd Desc'].str.replace("]","")

<ipython-input-253-c73b78b97a1c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("[","")
<ipython-input-253-c73b78b97a1c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("]","")
<ipython-input-253-c73b78b97a1c>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("[","")
<ipython-input-253-c73b78b97a1c>:7: FutureWarning: The default value of regex will change from True to False 

In [161]:
# ddf['Crm Cd Desc'].value_counts()

In [226]:
# ddf.columns

In [254]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    qr = '''CREATE TABLE crimedataFinal(



    
    crdate_occured timestamp , 
    crdrime_count integer,
    crhour integer,
    crarea varchar(500),
    crcrime_description varchar(1000),
    crvictim_age float,
    crprimise_descendent varchar(1000),
    crweapon_used float,
    crmale float,
    crfemale float,
    crvictDesc_black float,
    crvictDesc_white float,
    crvictDesc_asian float,
    crvictDesc_hispanic float,
    crvictDesc_others float,
    crinvestigation float,
    cradult_others float,
    cradult_arrest float,
    crjuvenile_arrest float,
    cryear0_18 float,
    cryear19_30 float,
    cryear31_45 float,
    cryear46_60 float,
    cryear60abv float
    
             );
    '''
    dbCursor.execute(qr)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL step 2", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()

In [153]:
# df['Premis Desc']=df['Premis Desc'].str.replace('\'','')

In [154]:
# df.shape

In [155]:
# df.iloc[17067]

In [255]:
ddf.shape

(4353, 24)

In [256]:
tpls = [tuple(x) for x in ddf.to_numpy()]

In [257]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()    
    print('done')

#     tpls = [tuple(x) for x in df.to_numpy()]
    sql32 = '''INSERT INTO crimedataFinal( crdate_occured, 
    crdrime_count,
    crhour,
    crarea,
    crcrime_description,
    crvictim_age,
    crprimise_descendent,
    crweapon_used,
    crmale,
    crfemale,
    crvictDesc_black,
    crvictDesc_white,
    crvictDesc_asian,
    crvictDesc_hispanic,
    crvictDesc_others,
    crinvestigation,
    cradult_others,
    cradult_arrest,
    crjuvenile_arrest,
    cryear0_18,
    cryear19_30,
    cryear31_45,
    cryear46_60,
    cryear60abv) 
       VALUES('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',
       '{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');'''

    for num,row in enumerate(tpls):
        # print("Data inserted using execute_many() successfully...")
        rw = sql32.format(*row)
        dbCursor.execute(rw)

    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
    print(num,row)
    print(rw)
finally:
    if dbConnection in locals(): 
        dbConnection.close()

done


In [212]:
# ddf['Crm Cd Desc'].value_counts()

In [258]:
sql = """
    SELECT
      *
    FROM
      crimedataFinal
    ;"""

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap"
    )
    dfs = sqlio.read_sql_query(sql, dbConnection)
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()

In [259]:
dfs.shape

(4353, 24)

In [260]:
ddf.to_csv("final_crime.csv",index=False)

In [261]:
ddf = pd.read_csv("final_crime.csv")

In [262]:
dfs.head()

,crdate_occured,crdrime_count,crhour,crarea,crcrime_description,crvictim_age,crprimise_descendent,crweapon_used,crmale,crfemale,crvictdesc_black,crvictdesc_white,crvictdesc_asian,crvictdesc_hispanic,crvictdesc_others,crinvestigation,cradult_others,cradult_arrest,crjuvenile_arrest,cryear0_18,cryear19_30,cryear31_45,cryear46_60,cryear60abv
0,2010-01-01,2304,8,THEFT OF IDENTITY,THEFT OF IDENTITY,35.0,THEFT OF IDENTITY,1.0,0.418403,0.546441,0.156684,0.183160,0.018663,0.501736,0.093316,0.831163,0.107205,0.057292,0.004340,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,BATTERY - SIMPLE ASSAULT,BATTERY - SIMPLE ASSAULT,35.0,BATTERY - SIMPLE ASSAULT,1.0,0.476636,0.414953,0.155140,0.244860,0.020561,0.357009,0.091589,0.824299,0.084112,0.085981,0.005607,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,BATTERY - SIMPLE ASSAULT,BATTERY - SIMPLE ASSAULT,35.0,BATTERY - SIMPLE ASSAULT,1.0,0.459108,0.429368,0.156134,0.263941,0.011152,0.373606,0.078067,0.832714,0.065056,0.096654,0.005576,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,BATTERY - SIMPLE ASSAULT BURGLARY FROM VEHICLE,BATTERY - SIMPLE ASSAULT BURGLARY FROM VEHICLE,35.0,BATTERY - SIMPLE ASSAULT BURGLARY FROM VEHICLE,1.0,0.439856,0.448833,0.156194,0.247756,0.023339,0.351885,0.087971,0.856373,0.070018,0.070018,0.003591,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,BURGLARY,BURGLARY,35.0,BURGLARY,1.0,0.440585,0.446069,0.168190,0.288848,0.023766,0.308958,0.084095,0.840951,0.085923,0.069470,0.003656,35.0,127.0,239.0,106.0,40.0
